In [1]:
import pandas
import numpy as np

In [2]:
df = pandas.read_csv('passwords.csv')
df = df.dropna(subset=['password'])

In [3]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

label_encoder = LabelEncoder()
df['strength'] = label_encoder.fit_transform(df['strength'])

onehot_encoder = OneHotEncoder(sparse_output=False)
strength_encoded = onehot_encoder.fit_transform(df['strength'].values.reshape(-1, 1))

In [4]:
max_length = 32 
X = df['password'].apply(lambda x: [ord(c) for c in x])
X = X.apply(lambda x: np.pad(x, (0, max_length - len(x)), 'constant') if len(x) < max_length else x[:max_length])
X = np.array(X.tolist())

In [5]:
import gc
del df
del onehot_encoder
del LabelEncoder
del OneHotEncoder
del pandas

gc.collect()

0

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, strength_encoded, test_size=0.2, random_state=1441)

In [7]:
del X
del strength_encoded

gc.collect()

0

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

model = Sequential()
model.add(Dense(256, input_dim=max_length, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(5, activation='softmax'))

2024-12-20 13:43:41.990246: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-20 13:43:42.051881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734689622.093982  360141 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734689622.097514  360141 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 13:43:42.112356: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [9]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
model.fit(X_train, y_train, epochs=10, batch_size=256, validation_data=(X_test, y_test))

Epoch 1/10
44827/44827 ━━━━━━━━━━━━━━━━━━━━ 72s 2ms/step - accuracy: 0.9401 - loss: 0.2313 - val_accuracy: 0.9410 - val_loss: 0.2217
Epoch 2/10
44827/44827 ━━━━━━━━━━━━━━━━━━━━ 77s 2ms/step - accuracy: 0.9401 - loss: 0.2306 - val_accuracy: 0.9415 - val_loss: 0.2274
Epoch 3/10
44827/44827 ━━━━━━━━━━━━━━━━━━━━ 70s 2ms/step - accuracy: 0.9404 - loss: 0.2291 - val_accuracy: 0.9419 - val_loss: 0.2185
Epoch 4/10
44827/44827 ━━━━━━━━━━━━━━━━━━━━ 62s 1ms/step - accuracy: 0.9402 - loss: 0.2295 - val_accuracy: 0.9423 - val_loss: 0.2269
Epoch 5/10
44827/44827 ━━━━━━━━━━━━━━━━━━━━ 63s 1ms/step - accuracy: 0.9405 - loss: 0.2295 - val_accuracy: 0.9431 - val_loss: 0.2205
Epoch 6/10
44827/44827 ━━━━━━━━━━━━━━━━━━━━ 66s 1ms/step - accuracy: 0.9403 - loss: 0.2301 - val_accuracy: 0.9425 - val_loss: 0.2199
Epoch 7/10
44827/44827 ━━━━━━━━━━━━━━━━━━━━ 68s 2ms/step - accuracy: 0.9403 - loss: 0.2302 - val_accuracy: 0.9431 - val_loss: 0.2194
Epoch 8/10
44827/44827 ━━━━━━━━━━━━━━━━━━━━ 64s 1ms/step - accuracy: 

In [55]:
def predict_password_strength(password):
    if len(password) > max_length:
        password = password[:max_length]
    
    password_encoded = np.array([ord(c) for c in password]).reshape(1, -1)
    password_encoded = np.pad(password_encoded, ((0, 0), (0, max_length - password_encoded.shape[1])), 'constant')
    prediction = model.predict(password_encoded)
    predicted_class = np.argmax(prediction)
    return label_encoder.inverse_transform([predicted_class])[0]

In [56]:
password = 'aghabidareh'
print(len(password))
strength = predict_password_strength(password)
print(f"Password Strength: {strength}")

11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Password Strength: good


In [57]:
model.save('model.h5')

In [8]:
import pickle
with open('encoder.h5' , 'wb') as f:
    pickle.dump(label_encoder , f)